# 1. 설치

In [8]:
from selenium import webdriver
driver = webdriver.Chrome() #웹드라이버 객체생성

# 2. 기초 조작법

In [9]:
driver.get('https://www.naver.com') #원하는 창 이동

driver.back() #뒤로가기
driver.forward() #앞으로 가기
driver.refresh() #새로고침

driver.current_url #현재 url 찾기 #리다이렉션(URL강제이동) 시, 최종URL을 확인할 때 유용

driver.title #창 title 태그

driver.maximize_window() #최대화
driver.minimize_window() #최소화

# driver.close() #현재창 닫기
# driver.quit() #창닫고, driver 종료

# 3. 기본템플릿
- CSS 선택자로 태그찾기
- send_keys 활용법 > WebElement 객체에 텍스트 입력, 키보드 입력

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

driver = webdriver.Chrome()

driver.get("https://www.naver.com")

srch = driver.find_element(By.CSS_SELECTOR, '#query') #반환된 WebElement 객체를 srch에 할당

srch.click()
srch.send_keys("스타트코딩")
srch.send_keys(Keys.ENTER)

# 4. 실전 테크닉

## 1) 여러태그 동시에 제어하기
- 공통 부모태그를 이용하여 'CSS선택자'를 '직접' 생성해준다. (기존 우클릭>copy는 단 하나에만 적용 가능)
- find_elements로 객체 '리스트'를 가져옴
- 반복문을 돌면서 click

In [ ]:
#카테고리, 브랜드의 체크박스를 여러개 체크해보기
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

driver = webdriver.Chrome()

driver.get("https://startcoding.pythonanywhere.com/basic")

labels = driver.find_elements(By.CSS_SELECTOR,'label[for]') #.input-checkbox > label 도 가능

import time

for label in labels:    
    print(label.text, label.get_attribute('for'))
    label.click()
    time.sleep(1)

    #이때, 너무 빠른 속도의 자동화는 문제가 생길 수 있음.


## 2) 동적대기 기법
- 웹페이지 태그가 즉시 로드되지 않고, 일정 동작이나 시간 이후에 나타나는 경우를 처리
- implicit Wait() : 암시적 대기, 정한 시간 내에 태그가 있으면 바로 이동, 웹 드라이버 객체에 대해 한번 설정하면 객체가 살아있는 동안은 계속 유효
- WebDriverWait, expected_conditions : 명시적 대기, 특정 조건을 만족할 때까지 시도, 웹 드라이버 객체에 대해 한번 설정하면 객체가 살아있는 동안은 계속 유효

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

driver = webdriver.Chrome()

driver.get("https://www.naver.com")

 #암시적 대기
driver.implicitly_wait(10)
driver.find_element(By.CSS_SELECTOR, '#query')

#명시적 대기
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

element = WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((By.CSS_SELETOR, '#query'))
)

## 3) 고급입력 컨트롤

In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

driver = webdriver.Chrome()

driver.get("https://www.naver.com")


In [ ]:

# 입력값 삭제 > 객체.clear
search = driver.find_element(By.CSS_SELECTOR, '#query')
search.send_keys('스타트코딩')
search.clear()

# (중요) 순차적 키 입력 > 객체.send_keys()
search.send_keys('스타트코딩')
search.send_keys(Keys.CONTROL, "a")
search.send_keys(Keys.CONTROL, "c")
search.send_keys(Keys.CONTROL, "v")
search.clear()

search.send_keys('스타트코딩', Keys.ENTER)

#tab으로 텍스트박스간 이동도 가능함
search.send_keys('검색어', Keys.TAB, '검색어2')


## 4) 셀렉트박스 조작
- SELECT 태그로 구성 > 자식 태그로 option 태그, value 속성이 있음
- 주의
    - CSS 선택자(option태그)로 선택하게 되면 태그의 위치가 바뀌면 동작하지 않을 수 있음, 가독성, 유지보수 어려움
    - 대신, Select 클래스로 사용
- 단계
    - 1) WebElement 객체 찾기
    - 2) WebElement 객체를 Select 객체로 변환시키기
    - 3) Select 객체로 원하는 값을 선택

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By

driver = webdriver.Chrome()

driver.get("https://finance.naver.com/research/company_list.naver")

In [6]:
# select 태그찾기 #contentarea_left > form > fieldset > ul > li:nth-child(2) > select
select_tag = driver.find_element(By.CSS_SELECTOR, '#contentarea_left > form > fieldset > ul > li:nth-child(2) > select')

from selenium.webdriver.support.select import Select
# Select 객체로 변환 : Select(select_tag)
# 인덱스로 선택 : 시작번호는 0
Select(select_tag).select_by_index(2)
# value 속성값으로 선택 : 언제나 문자열로 데이터 전달
Select(select_tag).select_by_value('40')
# 보이는 텍스트로 선택
Select(select_tag).select_by_visible_text('다올투자증권')

## 5) (중요) 여러페이지 관리방법
- 페이지 안에 다른 페이지가 있는 경우 (iframe 제어방법)
    - 웹드라이버는 한번에 하나의 페이지만 제어할 수 있음 ***
    - 서브페이지를 바라보도록 드라이버를 전환 시킨 후, 서브페이지 제어
    - 1. iframe 태그를 찾는다. 2. iframe 태그 객체 생성 3. 드라이버 전환 4.(필요시) 원래 페이지로 전환

- 새로운 창 제어(창이 여러개 뜰 때)
    1. 현재 드라이버가 가지고 있는 창 목록을 리스트 형태로 반환
    2. 새로운 창으로 전환
